In [ ]:
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt 
import numpy as np 
import os 
import pandas as pd
import nltk
import seaborn as sns
import warnings
from textblob import TextBlob
import os
import json
import gzip
import ast

In [ ]:
!pip install scikit-surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 KB 13.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp38-cp38-linux_x86_64.whl size=3366564 sha256=405dedd3ca31314ee2b633d9c1d8d48131cf091b9485efe700adf4c9c76b7ec3
  Stored in directory: /root/.cache/pip/wheels/af/db/86/2c18183a80ba05da35bf0fb7417aac5cddbd93bcb1b92fd3ea
Successfully built scikit-surprise


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
toy_df = pd.read_csv('/content/gdrive/Shareddrives/RS/LA02/toyset_supermarket.csv', sep=',', engine='python')

In [ ]:
toy_df=toy_df.drop(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1'], axis=1)

In [ ]:
toy_df.head()

,rating,reviewerName,reviewText,categories,gPlusPlaceId,unixReviewTime,reviewTime,gPlusUserId,sentiment
0,4.0,william holcomb,"I moved to Tory last year,and started shopping...",['Supermarket'],104820477387020653006,1.361646e+09,"Feb 23, 2013",100000996709496791884,positive
1,2.0,Joseph Black,needs new emploies and a remodel,"['Supermarket', 'Department Store']",101364495924446646306,1.355618e+09,"Dec 15, 2012",100001643728576657268,positive
2,4.0,Hugo Villegas,good music. very friendly janitors,"['Supermarket', 'Bakery']",114324634877118350105,1.356147e+09,"Dec 21, 2012",100001874772599022260,positive
3,4.0,Mitesh Shanbhag,Don't be fooled by the shopfront - it's like e...,['Chinese Supermarket'],111601036200348057782,1.364428e+09,"Mar 27, 2013",100002870366309555032,positive
4,5.0,quicksand senx,"As the Fousquare mayor, all I have to say is: ...",['Supermarket'],116317152167129429134,1.385472e+09,"Nov 26, 2013",100003258120679643845,positive


In [ ]:
toy_df['gPlusUserId'].nunique()

18065

In [ ]:
toy_df['rating'].value_counts()

5.0    9961
4.0    5464
3.0    2680
2.0    1220
1.0     850
Name: rating, dtype: int64

In [ ]:
rating_mat = toy_df.pivot(index="gPlusUserId", columns="gPlusPlaceId", values="rating").fillna(0)

In [ ]:
rating_mat

gPlusPlaceId,100000007848517518160,100000953751536585127,100003361583829028219,100003877831445093712,100004127083751659268,100004409044316734159,100004742840026123700,100004901418529042986,100008161235723075350,100010582906983326187,...,118421643604653891541,118422207418654273872,118424586425714331424,118424906674795214467,118427571303334817183,118432371535187053758,118432625845762640497,118435641356113847202,118443084163335841298,118443464138186193706
gPlusUserId,,,,,,,,,,,,,,,,,,,,,
100000996709496791884,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100001643728576657268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100001874772599022260,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100002870366309555032,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100003258120679643845,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118442241398749352779,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
118443144244237053967,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
118443910897342529434,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# # Predict the ratings for each user-item pair using the dot product of U, sigma, and Vt
# all_user_predicted_ratings = np.dot(np.dot(u[:,:15], np.diag(s[:15])), vh[:15,:])

In [ ]:
# # Print the predicted ratings matrix
# print('Predicted ratings matrix:')
# print(all_user_predicted_ratings)

# # Define a function to recommend items for a given user
# def recommend_items(user_id, predicted_ratings, num_recommendations=15):
#     # Get the user's predicted ratings
#     user_ratings = predicted_ratings[user_id, :]
#     # Sort the ratings in descending order and return the top n items
#     sorted_ratings = user_ratings.argsort()[::-1]
#     top_n_items = sorted_ratings[:num_recommendations]
#     return top_n_items

In [ ]:
import numpy as np
from scipy.sparse.linalg import svds
matrix = rating_mat.values

# Convert the data type of the rating matrix to float
matrix = matrix.astype(float)

# Apply SVD on the rating matrix to get the latent factors
U, sigma, Vt = svds(matrix, k=10)
sigma = np.diag(sigma)

# Predict the ratings for each user-item pair using the dot product of U, sigma, and Vt
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt)

# Print the predicted ratings matrix
# print('Predicted ratings matrix:')
# print(all_user_predicted_ratings)

# Define a function to recommend items for a given user
def recommend_items(user_id, predicted_ratings, num_recommendations=10):
    # Get the user's predicted ratings
    user_ratings = predicted_ratings[user_id]
    # Sort the ratings in descending order and return the top n items
    sorted_ratings = user_ratings.argsort()[::-1]
    top_n_items = sorted_ratings[:num_recommendations]
    return top_n_items


In [ ]:
# Test the recommendation function for a sample user
user_id = 0
recommendations = recommend_items(user_id, all_user_predicted_ratings, num_recommendations=10)
print(f'Recommended items for user {user_id}: {recommendations}')

Recommended items for user 0: [7893 3586 2200 6596 7846 1831  122 1528 7247 9397]


In [ ]:
map_place = pd.read_csv('/content/gdrive/Shareddrives/RS/LA02/places_name.csv', sep=',', engine='python')

In [ ]:
map_place.set_index('gPlusPlaceId', inplace=True)

In [ ]:
for i in recommendations:
  item = rating_mat.columns[i]
  name = map_place.loc[item, 'name']
  print(name)

Whole Foods Market
Trader Joe's
Fairway Market
Mitsuwa Marketplace
Whole Foods Market
Whole Foods Market
Perelandra Natural Food Center
Western Beef Supermarket
Whole Foods Market
Key Food


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import io
from numpy.linalg import svd


# Define a set of k values
k_values = [1, 5, 10, 20, 50, 100, 200]

# Convert the data type of the rating matrix to float
matrix = matrix.astype(float)



# Compute the explained variance for each k value
explained_variances = []
for k in k_values:
    U, sigma, Vt = svds(matrix, k)
    explained_variances.append(np.sum(sigma[:k]) / np.sum(sigma))

In [ ]:
# fig, ax = plt.subplots()
# ax.plot(k_values, explained_variances, marker='o')
# ax.set_xlabel('k')
# ax.set_ylabel('Explained Variance')
# ax.set_title('Explained Variance vs. k')
# plt.show()

#Visualizations

In [ ]:
import plotly.graph_objects as go
rating_array = toy_df['rating'].tolist()
rating_array = np.array(rating_array)
rating_array
fig = go.Figure(data=[go.Histogram(x=rating_array, histnorm='probability', marker_color='magenta', opacity=0.75)])
fig.update_layout(
    title_text='The ratio of ratings for supermarket category', # title of plot
    xaxis_title_text='Ratings', # xaxis label
    yaxis_title_text='Ratio', # yaxis label)
)

fig.show()